In [ ]:
import sys
import os
import numpy as np
import dill
from cycler import cycler
%matplotlib inline
import matplotlib.pyplot as plt
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)
from src.utilities.AttrDict import AttrDict
GEOMETRY = AttrDict()
GATE = AttrDict()
from src.configuration import t#, dt, n_x, n_z, rho_s, dx, dz
# from scipy.interpolate import interp1d, InterpolatedUnivariateSpline
from src.utilities.nearest import nearest
from src.initialisation.initialise_main import initialisation
from src.pressure import pressure
from src.stress import stress_time
from src.fatigue import fatigue

# Plotting options
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Helvetica'] + plt.rcParams['font.serif']
params = {'legend.fontsize': 'large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)
plt.rcParams['axes.prop_cycle'] = cycler(color=['#000000'])

Specify the name of the load case and the amount of modes to consider, and initialize the model.
For new gate designs the entire notebook should be reset and re-initialized.

In [ ]:
GATE.case   = 'Gate1_test'
GATE.n_modes= 16
GATE.Ly     = 1 # Overhang length
GATE.zeta   = 0.02 # Damping
GATE.cdamp  = 0 # Distributed damping
GATE.HEIGHT = 7.5
GATE.WIDTH  = 10
## Heights of horizontal stiffeners
GEOMETRY.Height  = 2.55
GEOMETRY.Height2 = 5.0
GEOMETRY.Height3 = GATE.HEIGHT
## X-coordinates of vertical stiffeners
GEOMETRY.Width  = 2.5
GEOMETRY.Width2 = 5
GEOMETRY.Width3 = 7.45
GEOMETRY.Width4 = GATE.WIDTH
## Stiffener depths
GEOMETRY.LengthHorRibs  = 0.71
GEOMETRY.LengthVertRibs = 0.71
## Supports (Left-Bottom-Right-Top)
GEOMETRY.Stiffness  = 3e11
GEOMETRY.Stiffness2 = 0
GEOMETRY.Stiffness3 = 3e11
GEOMETRY.Stiffness4 = 0
## Element thicknesses
GEOMETRY.Platethickness    = 0.05
GEOMETRY.RibThicknessHor   = 0.1
GEOMETRY.RibThicknessHor2  = 0.1
GEOMETRY.RibThicknessHor3  = 0.1
GEOMETRY.RibThicknessHor4  = 0.1
GEOMETRY.RibThicknessVert  = 0.1
GEOMETRY.RibThicknessVert2 = 0.1
GEOMETRY.RibThicknessVert3 = 0.1
GEOMETRY.RibThicknessVert4 = 0.1
GEOMETRY.RibThicknessVert5 = 0.1
GATE.GEOMETRY = GEOMETRY

GATE = initialisation(GATE, overwrite=False)

# Load case definition
Probability distributions are derived from historical data and condensed into discrete load cases.

First the raw data is read from the specified text files and formatted in a DataFrame. The formatted data is then stored in the 02_preprocessing folder.

In [ ]:
from src.loadcases import import_raw_data, create_pdf, binning, binfilter
fig, raw_data = import_raw_data(windfile='KNMI_20201124_hourly.txt', waterfile='20201105_007.csv', bins=25)
fig

Next, the data is pre-processed and transformed into two probability distributions which also account for climate change.

In [ ]:
fig, dists = create_pdf(raw_data, rise=1, n_h=1e4, n_u=1e4, h_range=[2,10])
fig

The probability distributions are then integrated into discrete sections, whose expected values will represent the properties of the discrete load cases. The probability of occurrence is found by integrating the probability density over the area of the segment.

In [ ]:
with open('../data/03_loadevents/pdfs.cp.pkl', 'rb') as file:
    dists = dill.load(file)

In [ ]:
fig1, u_bins, h_bins, cases = binning(dists, h_res=0.1, u_res=1)
fig1

The load cases are then filtered based on their probability of occurrence and their intensity.

In [ ]:
fig2, filtcases = binfilter(cases, GATE, freqfilter=True, intensityfilter=True)
fig2

## Discrete load case resolution
An analysis of the effect of different load case resolutions on the variance of the response.

In [ ]:
from src.loadcases import sensitivity_discretisation

h_res_list = [0.001, 0.1, 0.5, 1, 1.5]
u_res_list = [0.1, 1, 2, 5, 10, 20]
h_sea = 6
u_wind = 40
fig = sensitivity_discretisation(h_sea, h_res_list, u_wind, u_res_list, runs=250, coords=(5,1,7.5), overwrite=False)
fig

# Fatigue over gate surface
Calculating the fatigue due to a given load event at every point on the gate's surface to identify the distribution and critical coordinates.

The next cell plots the fatigue across the gate for one or multiple gate designs, and determines the coordinate where the most fatigue occurs.

In [ ]:
from src.full_gate_fatigue import fatigue_gate, plot_fatigue_gate, modalstresscontribution, modalfatiguecontribution

u_wind = 20
h_sea = 7
GATE, damage_gate, modeshare = fatigue_gate(u_wind, h_sea, cat=100, ID=GATE.case, overwrite=False)
# plot_fatigue_gate(['Gate1_test', 'Gate1_test_1mode'], ['16 modes', '1 mode'], u_wind, h_sea)
plot_fatigue_gate(['Gate1_test'], ['16 modes'], u_wind, h_sea)

## Mode contribution
Various ways of determining the relative importance of different modes on the total response across the gate.

In [ ]:
import matplotlib.colors as colors
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

def plot_modeshare_gate_6(modes, modeshare):
    """Plots the contribution of mode(s) to the total fatigue at every gate coordinate.

    Parameters:
    modes: The mode(s) whose contribution to fatigue should be plotted (-)
    modeshare: List of mode contributions for every gate coordinate (%)

    """

    # Determine % of stress caused by specified modes
    fig = plt.figure(figsize=[15,25])

    for m, mode in enumerate(modes): 
        share = np.array([point[mode-1]/sum(point)*100 for point in modeshare])
        ax = fig.add_subplot(3, 2, m+1, projection='3d')
        Zmin = 0
        Zmax = 100
        cmap = plt.cm.Reds
        norm = colors.Normalize(Zmin,Zmax)

        coords = []
        response = []
        for face in GATE.faces:
            coords.append(GATE.coords[face-1])
            response.append(share[face-1].mean())

        facets = Poly3DCollection(coords)
        facets.set_facecolor(cmap(norm(response)))
        facets.set_edgecolor('dimgray')
        ax.add_collection3d(facets)

        cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), fraction=0.03, pad=.1)
        cbar.set_label("Share of stress response [%]", rotation=270, labelpad=10)

        plotmodes = ', '.join(map(str, modes))
        ax.set_title('Mode %s ($f_{dry}=%s$ Hz)'%(mode, round(GATE.dry_freqs[mode-1],1)))
        ax.set_xlabel('X [m]')
        ax.set_ylabel('Y [m]')
        ax.set_zlabel('Z [m]')
        ax.set_xlim3d(0, GATE.WIDTH)
        ax.set_ylim3d(-4,4)
        ax.set_zlim3d(0,8)
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.xaxis.pane.set_edgecolor('w')
        ax.yaxis.pane.set_edgecolor('w')
        ax.zaxis.pane.set_edgecolor('w')
        ax.set_box_aspect((GATE.WIDTH, 8, 7.5))
        ax.view_init(30, 50)
        plt.close(fig)
    return fig

plot_modeshare_gate_6([1,2,3,6,13,14], modeshare)

The contribution of the different modes to the total stress in the gate is plotted. The leftmost plot shows the distribution across all gate coordinates and the rightmost one the distribution at the critical coordinate.

In [ ]:
modalstresscontribution(modeshare, max_coords)

This plot shows the effect on the fatigue when a given mode is removed from the response. A value of 60% means that 60% of the fatigue is lost (does not add up to 100%).

In [ ]:
fig, shares = modalfatiguecontribution(u_wind, h_sea, max_coords, cat=100, save=False)
fig

This plot shows the combined contribution of a given set of modes to the total stress at every coordinate.

In [ ]:
from src.full_gate_fatigue import plot_modeshare_gate
modes = [1]
plot_modeshare_gate(modes, modeshare)

# Full lifetime fatigue analysis
This section uses the load cases from Section 1 to compute a probabilistic estimate of the lifetime fatigue

First, define the coordinates to evaluate and the name under which the results should be stored.

In [ ]:
coords = GATE.max_coords
version = GATE.case
print(coords, version)

The following function evaluates the fatigue for all previously derived load cases at the coordinate specified above, and performs a probabilistic fatigue lifetime simulation (can take an hour for new cases). If the version name has already been used before, the existing data will be loaded instead.

In [ ]:
from src.lifetime import generate_fatigue, simulations

damage_list = generate_fatigue(coords, version, N=1e6)
fig, D_expected, totals, [maxes,means,mins] = simulations(runs=1000, version=version, average_only=False)

# ULS check
Use the same probabilistic model to perform a ULS-check for a normative event.

This cell derives the normative load event based on 1/10.000 year wind conditions and a water level equal to the overhang, based on the environmental data. Can also be manually defined.

In [ ]:
import scipy.stats as ss
h_sea = GATE.HEIGHT

with open('../data/03_loadevents/u_dist.pkl', 'rb') as f:
    fit = dill.load(f)
prob = (10000*365.25*24)**-1
u_wind = ss.lognorm(*fit).ppf(1-prob)
_,_,_,Hs,_ = pressure(u_wind, h_sea)
print("Esimated once per 10.000 years wind velocity condition is "+
      str(round(u_wind,2))+'m/s (Hs='+str(round(Hs,2))+'m).')

with open('../data/03_loadevents/pdfs.cp.pkl', 'rb') as file:  
    x_h, hcc_pdf, x_u, u_pdf = dill.load(file)
res = 0.1
slice_h = (x_h > h_sea-res/2) & (x_h < h_sea+res/2)
print("Combined probability of %s"%round(np.trapz(hcc_pdf[slice_h], x=x_h[slice_h])/10000, 10))

In [ ]:
from src.analyses.uls_simulations import uls_simulations, uls_plot
runs = 1000
f_yd = 355
version='1'

max_stresses = uls_simulations(h_sea, u_wind, GATE.max_coords, runs, version=version)
fig = uls_plot(runs, 355, version)
fig